In [3]:
import numpy as np
import pandas as pd


In [4]:
data=pd.read_csv("C:/Users/SOUMODIP/Downloads/archive (1)/TCS Historical Data.csv")
data.head()

,Unnamed: 0,Date,Price,Open,High,Low,Volume,Change(%)
0,0,09 Oct 2024,4252.95,4260.0,4295.00,4246.70,1082504,-0.01%
1,1,08 Oct 2024,4253.25,4272.6,4279.40,4203.95,1541867,-0.46%
2,2,07 Oct 2024,4272.85,4273.9,4297.25,4229.40,1472619,0.48%
3,3,04 Oct 2024,4252.25,4225.0,4298.00,4224.05,2965463,0.46%
4,4,03 Oct 2024,4232.75,4262.6,4279.00,4210.00,3350119,-1.29%


In [ ]:
# data set should look like this
#Date | Open | High | Low | Close | Volume



In [8]:
# 1. Drop useless columns
if "Unnamed: 0" in data.columns:
    data = data.drop(columns=["Unnamed: 0"])
if "Change(%)" in data.columns:
    data = data.drop(columns=["Change(%)"])
# 2. Rename Price -> Close
data = data.rename(columns={"Price": "Close"})
# 3. Convert Date to datetime
data["Date"] = pd.to_datetime(data["Date"])
# 4. Sort data OLD -> NEW
data = data.sort_values("Date").reset_index(drop=True)
# 5. Convert numeric columns
numeric_cols = ["Open", "High", "Low", "Close", "Volume"]
data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors="coerce")
# 6. Drop rows with missing values
data = data.dropna(subset=numeric_cols)
# Final sanity check
print(data.head())
print(data.tail())
print(data.shape)


        Date    Close    Open     High     Low   Volume
0 2019-10-14  2021.00  1995.0  2032.75  1968.0  3510704
1 2019-10-15  2037.30  2032.0  2047.00  2022.7  2261310
2 2019-10-16  2046.40  2038.0  2056.00  2016.0  3034082
3 2019-10-17  2030.95  2014.0  2038.00  1999.3  3818774
4 2019-10-18  2057.35  2037.5  2063.90  2025.0  3087901
           Date    Close    Open     High      Low   Volume
1236 2024-10-03  4232.75  4262.6  4279.00  4210.00  3350119
1237 2024-10-04  4252.25  4225.0  4298.00  4224.05  2965463
1238 2024-10-07  4272.85  4273.9  4297.25  4229.40  1472619
1239 2024-10-08  4253.25  4272.6  4279.40  4203.95  1541867
1240 2024-10-09  4252.95  4260.0  4295.00  4246.70  1082504
(1241, 6)


In [7]:
# data is already cleaned and sorted OLD -> NEW
# Columns: Date, Open, High, Low, Close, Volume
df = data.copy()
# RETURNS (momentum)
df["ret_1d"] = df["Close"].pct_change(1)
df["ret_5d"] = df["Close"].pct_change(5)
# MOVING AVERAGE DISTANCE
df["ma_10"] = df["Close"].rolling(10).mean()
df["ma_dist_10"] = (df["Close"] - df["ma_10"]) / df["ma_10"]
# VOLATILITY
df["volatility_10d"] = df["ret_1d"].rolling(10).std()
# VOLUME FEATURES
df["vol_chg_1d"] = df["Volume"].pct_change(1)
df["vol_ma_10"] = df["Volume"].rolling(10).mean()
df["vol_ratio_10"] = df["Volume"] / df["vol_ma_10"]
# TARGET (5-day forward return)
df["target_5d"] = df["Close"].shift(-5) / df["Close"] - 1
# FINAL CLEANUP
feature_cols = [
    "ret_1d", "ret_5d", "ma_dist_10",
    "volatility_10d", "vol_chg_1d", "vol_ratio_10"
]
df_model = df.dropna(subset=feature_cols + ["target_5d"])

print(df_model.head())
print(df_model.tail())
print(df_model.shape)


         Date    Close    Open     High      Low   Volume    ret_1d    ret_5d  \
10 2019-10-29  2194.85  2121.5  2201.70  2121.50  5064173  0.037411  0.069928   
11 2019-10-30  2252.80  2225.0  2260.00  2176.85  5195136  0.026403  0.088257   
12 2019-10-31  2269.65  2250.0  2284.95  2237.00  4879261  0.007480  0.090130   
13 2019-11-01  2200.90  2264.0  2275.00  2195.00  4172620 -0.030291  0.035742   
14 2019-11-04  2193.95  2207.0  2229.20  2180.90  2646411 -0.003158  0.036985   

       ma_10  ma_dist_10  volatility_10d  vol_chg_1d  vol_ma_10  vol_ratio_10  \
10  2081.100    0.054659        0.013250   21.565504  3214421.6      1.575454   
11  2102.650    0.071410        0.014421    0.025861  3507804.2      1.481022   
12  2124.975    0.068083        0.014319   -0.060802  3692322.1      1.321461   
13  2141.970    0.027512        0.018654   -0.144825  3727706.7      1.119353   
14  2155.630    0.017777        0.018892   -0.365768  3683557.7      0.718439   

    target_5d  
10   0.003

In [9]:
df_model.to_csv("tcs_features_5d.csv", index=False)
